In [ ]:
import sys
from pathlib import Path
sys.path.append("../")

In [ ]:
import ipywidgets as wdgt
from pathlib import Path

# Set up transcript file selection
transcript_dir = Path('../data/transcripts')
transcript_files = {f.stem: str(f) for f in transcript_dir.glob('*.json')}

# Create and display dropdown widget
transcript_select = wdgt.Dropdown(
    options=transcript_files,
    description='Select Transcript:',
    layout=wdgt.Layout(width='75%')
)

# Initialize and update transcript path
transcript_path = Path(transcript_files[next(iter(transcript_files))])

def update_transcript_file(change) -> None:
    global transcript_path
    transcript_path = Path(change.new)
    print(f"Selected transcript: {transcript_path}")

transcript_select.observe(update_transcript_file, names='value')
display(transcript_select)


In [ ]:
import json
from collections import defaultdict
from typing import Dict, List, Tuple

# Load transcript data
with open(transcript_path) as f:
    transcript_data = json.load(f)

async def analyze_transcript(transcript_path: Path) -> Dict:
    """
    Analyze transcript content including speaker stats and timing information.
    
    Args:
        transcript_path: Path to the transcript JSON file
        
    Returns:
        Dict containing analysis results
    """

    
    # Initialize analysis containers
    speaker_stats = defaultdict(lambda: {
        "total_time": 0.0,
        "word_count": 0,
        "segments": 0
    })
    
    total_duration = 0
    
    # Analyze segments from the segments list in the JSON structure
    for segment in transcript_data["segments"]:
        speaker = segment["speaker"] 
        duration = segment["end"] - segment["start"]
        words = len(segment["words"])
        
        # Update speaker statistics
        speaker_stats[speaker]["total_time"] += duration
        speaker_stats[speaker]["word_count"] += words
        speaker_stats[speaker]["segments"] += 1
        
        total_duration = max(total_duration, segment["end"])
    
    # Calculate summary statistics
    analysis = {
        "total_duration": total_duration,
        "total_speakers": len(speaker_stats),
        "speaker_stats": dict(speaker_stats)
    }
    
    # Print summary
    print(f"\nTranscript Analysis Summary:")
    print(f"Total Duration: {total_duration:.2f} seconds")
    print(f"Number of Speakers: {len(speaker_stats)}")
    print("\nSpeaker Statistics:")
    
    for speaker, stats in speaker_stats.items():
        print(f"\n{speaker}:")
        print(f"  Speaking Time: {stats['total_time']:.2f} seconds")
        print(f"  Word Count: {stats['word_count']}")
        print(f"  Number of Segments: {stats['segments']}")
        if stats['total_time'] > 0:
            print(f"  Words per Second: {stats['word_count']/stats['total_time']:.2f}")
            
    return analysis

# Run analysis on selected transcript
analysis_results = await analyze_transcript(transcript_path)


In [ ]:
def format_timestamp(seconds: float) -> str:
    """Convert seconds to HH:MM:SS format"""
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60) 
    secs = int(seconds % 60)
    return f"{hours:02d}:{minutes:02d}:{secs:02d}"

from ipywidgets import HTML, VBox, Layout
from textwrap import fill

# Create formatted HTML output
html_output = ["<h3>Meeting Script - Combined by Speaker</h3>"]
html_output.append("<hr>")

current_speaker = None
current_text = []
current_start = None

for segment in transcript_data["segments"]:
    if current_speaker != segment["speaker"]:
        # Output previous speaker's text
        if current_speaker:
            timestamp = format_timestamp(current_start)
            wrapped_text = fill(' '.join(current_text), width=80)
            html_output.append(f"<p><b>[{timestamp}] {current_speaker}:</b><br>")
            html_output.append(f"{wrapped_text}</p>")
            html_output.append("<hr>")
            
        # Start new speaker
        current_speaker = segment["speaker"]
        current_text = [segment["text"].strip()]
        current_start = segment["start"]
    else:
        # Continue current speaker
        current_text.append(segment["text"].strip())

# Output final speaker
if current_speaker:
    timestamp = format_timestamp(current_start)
    wrapped_text = fill(' '.join(current_text), width=80)
    html_output.append(f"<p><b>[{timestamp}] {current_speaker}:</b><br>")
    html_output.append(f"{wrapped_text}</p>")
    html_output.append("<hr>")

# Display formatted output
display(HTML(
    value=''.join(html_output),
    layout=Layout(width='100%', border='1px solid gray', padding='10px')
))
